In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score

In [2]:
# Load dataset
file_path = '/content/ObesityDataSet_raw_and_data_sinthetic.csv'
csv = pd.read_csv(file_path)

In [4]:
print(csv.columns)

Index(['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight',
       'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE',
       'CALC', 'MTRANS', 'NObeyesdad'],
      dtype='object')


In [5]:
# Encode kolom target kategorikal 'NObeyesdad'
label_encoder = LabelEncoder()
csv['NObeyesdad'] = label_encoder.fit_transform(csv['NObeyesdad'])

# Identifikasi kolom kategorikal dan lakukan encoding
kolom_kategorikal = csv.select_dtypes(include=['object']).columns
for kolom in kolom_kategorikal:
    csv[kolom] = LabelEncoder().fit_transform(csv[kolom])

# Pisahkan fitur dan target
X = csv.drop(columns=['NObeyesdad'])  # Menyaring target
y = csv['NObeyesdad']  # Menetapkan target

In [6]:
# Isi nilai yang hilang dengan 0 dan normalisasi fitur
X = X.fillna(0)  # Mengganti nilai yang hilang dengan 0
scaler = StandardScaler()
X = scaler.fit_transform(X)  # Melakukan normalisasi pada fitur

# Membagi data menjadi set pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Mengonversi data menjadi tensor PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Memindahkan tensor ke GPU jika tersedia
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_train_tensor = X_train_tensor.to(device)
X_test_tensor = X_test_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

In [7]:
# Model RNN sederhana
class ModelRNN(nn.Module):
    def __init__(self, ukuran_input, ukuran_hidden, jumlah_kelas, jenis_pooling):
        super(ModelRNN, self).__init__()
        # Inisialisasi RNN dengan input dan hidden size, batch_first=True berarti input memiliki bentuk (batch, seq, input_size)
        self.rnn = nn.RNN(ukuran_input, ukuran_hidden, batch_first=True)  # RNN sederhana
        self.jenis_pooling = jenis_pooling  # Jenis pooling: 'max' atau 'avg'
        # Lapisan fully connected untuk menghasilkan output klasifikasi
        self.fc = nn.Linear(ukuran_hidden, jumlah_kelas)  # Lapisan fully connected untuk klasifikasi

    def forward(self, x):
        # Forward pass melalui RNN untuk mendapatkan output
        out, _ = self.rnn(x)  # Forward pass melalui RNN
        if self.jenis_pooling == 'max':
            # Pooling maksimum untuk memilih nilai tertinggi di setiap fitur
            out = torch.max(out, dim=1).values  # Pooling maksimum
        elif self.jenis_pooling == 'avg':
            # Pooling rata-rata untuk menghitung rata-rata nilai setiap fitur
            out = torch.mean(out, dim=1)  # Pooling rata-rata
        # Melalui lapisan fully connected untuk klasifikasi akhir
        out = self.fc(out)  # Lapisan klasifikasi akhir
        return out

In [8]:
# Model Deep RNN
class ModelDeepRNN(nn.Module):
    def __init__(self, ukuran_input, ukuran_hidden, jumlah_kelas, jenis_pooling, num_layers=2):
        super(ModelDeepRNN, self).__init__()
        # Inisialisasi Deep RNN dengan multi-layer, num_layers menentukan jumlah lapisan RNN
        self.rnn = nn.RNN(ukuran_input, ukuran_hidden, num_layers=num_layers, batch_first=True)  # Deep RNN
        self.jenis_pooling = jenis_pooling  # Jenis pooling: 'max' atau 'avg'
        # Lapisan fully connected untuk menghasilkan output klasifikasi
        self.fc = nn.Linear(ukuran_hidden, jumlah_kelas)  # Lapisan fully connected untuk klasifikasi

    def forward(self, x):
        # Forward pass melalui Deep RNN untuk mendapatkan output
        out, _ = self.rnn(x)  # Forward pass melalui Deep RNN
        if self.jenis_pooling == 'max':
            # Pooling maksimum untuk memilih nilai tertinggi di setiap fitur
            out = torch.max(out, dim=1).values  # Pooling maksimum
        elif self.jenis_pooling == 'avg':
            # Pooling rata-rata untuk menghitung rata-rata nilai setiap fitur
            out = torch.mean(out, dim=1)  # Pooling rata-rata
        # Melalui lapisan fully connected untuk klasifikasi akhir
        out = self.fc(out)  # Lapisan klasifikasi akhir
        return out

In [10]:
# Hyperparameter eksperimen
ukuran_input = X_train_tensor.shape[1]  # Jumlah fitur input (jumlah kolom fitur)
jumlah_kelas = len(label_encoder.classes_)  # Jumlah kelas output (jumlah kategori target)
ukuran_hidden_list = [16, 32]  # Daftar ukuran hidden layer yang akan diuji (jumlah unit dalam layer tersembunyi)
jenis_pooling_list = ['max', 'avg']  # Metode pooling yang akan diuji: 'max' untuk pooling maksimum, 'avg' untuk pooling rata-rata
daftar_epoch = [5, 50, 100, 250, 350]  # Daftar jumlah epoch yang akan diuji (jumlah iterasi training)
daftar_optimizer = ['SGD', 'RMSprop', 'Adam']  # Daftar optimizer yang akan diuji: SGD, RMSprop, atau Adam

# Variabel untuk menyimpan model terbaik
model_terbaik = None  # Menyimpan model terbaik berdasarkan performa
akurasi_terbaik = 0  # Menyimpan akurasi terbaik
parameter_terbaik = {}  # Menyimpan parameter terbaik yang menghasilkan model terbaik
ringkasan_hasil = []  # Untuk menyimpan ringkasan hasil eksperimen hyperparameter

In [11]:
# Eksperimen kombinasi hyperparameter
for model_type in ['RNN', 'DeepRNN']:
    # Loop untuk mengeksplorasi ukuran hidden layer yang berbeda
    for ukuran_hidden in ukuran_hidden_list:
        # Loop untuk mencoba berbagai metode pooling
        for jenis_pooling in jenis_pooling_list:
            # Loop untuk memilih optimizer yang berbeda
            for nama_optimizer in daftar_optimizer:
                # Loop untuk mencoba jumlah epoch yang berbeda
                for epoch in daftar_epoch:
                    print(f"--- Model: {model_type}, Ukuran Hidden: {ukuran_hidden}, Pooling: {jenis_pooling}, Optimizer: {nama_optimizer}, Epoch: {epoch} ---")

                    # Inisialisasi model berdasarkan jenis model (RNN atau DeepRNN)
                    if model_type == 'RNN':
                        model = ModelRNN(ukuran_input, ukuran_hidden, jumlah_kelas, jenis_pooling).to(device)
                    elif model_type == 'DeepRNN':
                        model = ModelDeepRNN(ukuran_input, ukuran_hidden, jumlah_kelas, jenis_pooling).to(device)

                    fungsi_loss = nn.CrossEntropyLoss()  # Menggunakan CrossEntropyLoss untuk klasifikasi multi-kelas

                    # Pilih optimizer sesuai dengan pilihan yang ada
                    if nama_optimizer == 'SGD':
                        optimizer = optim.SGD(model.parameters(), lr=0.01)
                    elif nama_optimizer == 'RMSprop':
                        optimizer = optim.RMSprop(model.parameters(), lr=0.01)
                    elif nama_optimizer == 'Adam':
                        optimizer = optim.Adam(model.parameters(), lr=0.01)

                    # Pengaturan Early Stopping untuk mencegah overfitting
                    batas_sabar = 10  # Batas jumlah epoch tanpa perbaikan pada loss validasi sebelum berhenti
                    loss_terbaik = float('inf')  # Inisialisasi nilai loss terbaik
                    penghitung_sabar = 0  # Penghitung untuk early stopping

                    # Loop pelatihan
                    for ep in range(epoch):
                        model.train()  # Mode pelatihan
                        optimizer.zero_grad()  # Bersihkan gradien sebelumnya
                        output = model(X_train_tensor.unsqueeze(1))  # Forward pass pada data latih
                        loss = fungsi_loss(output, y_train_tensor)  # Hitung loss
                        loss.backward()  # Backpropagation untuk menghitung gradien
                        optimizer.step()  # Perbarui parameter model

                        # Validasi setelah setiap epoch
                        model.eval()  # Mode evaluasi
                        with torch.no_grad():
                            val_output = model(X_test_tensor.unsqueeze(1))  # Forward pass pada data validasi
                            val_loss = fungsi_loss(val_output, y_test_tensor).item()  # Hitung loss validasi
                            prediksi_val = val_output.argmax(dim=1)  # Prediksi kelas
                            akurasi_val = accuracy_score(y_test_tensor.cpu(), prediksi_val.cpu()) * 100  # Hitung akurasi

                        # Log detail epoch (loss, val_loss, dan akurasi)
                        print(f"Epoch {ep + 1}/{epoch}, Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}, Akurasi: {akurasi_val:.2f}%")

                        # Cek Early Stopping: Jika loss validasi tidak berkurang, tambahkan penghitung sabar
                        if val_loss < loss_terbaik:
                            loss_terbaik = val_loss  # Update loss terbaik
                            penghitung_sabar = 0  # Reset penghitung sabar
                        else:
                            penghitung_sabar += 1  # Increment penghitung sabar

                        # Jika penghitung sabar mencapai batas, lakukan early stopping
                        if penghitung_sabar >= batas_sabar:
                            print("Early stopping diaktifkan.")
                            break

                    # Evaluasi model setelah pelatihan selesai
                    model.eval()
                    with torch.no_grad():
                        prediksi = model(X_test_tensor.unsqueeze(1)).argmax(dim=1)  # Prediksi pada test set
                        akurasi = accuracy_score(y_test_tensor.cpu(), prediksi.cpu())  # Hitung akurasi pada data uji

                    # Log akurasi akhir untuk kombinasi hyperparameter saat ini
                    print(f"Akurasi Akhir untuk Model: {model_type}, Ukuran Hidden: {ukuran_hidden}, Pooling: {jenis_pooling}, Optimizer: {nama_optimizer}, Epoch: {epoch}: {akurasi:.6f}")
                    ringkasan_hasil.append(f"{model_type},{ukuran_hidden},{jenis_pooling},{nama_optimizer},{epoch},{akurasi}")

                    # Simpan model dan parameter terbaik berdasarkan akurasi tertinggi
                    if akurasi > akurasi_terbaik:
                        akurasi_terbaik = akurasi  # Update akurasi terbaik
                        model_terbaik = model  # Simpan model terbaik
                        parameter_terbaik = {
                            'model_type': model_type,
                            'ukuran_hidden': ukuran_hidden,
                            'jenis_pooling': jenis_pooling,
                            'optimizer': nama_optimizer,
                            'epoch': epoch
                        }  # Simpan parameter terbaik

Streaming output truncated to the last 5000 lines.
Epoch 147/350, Loss: 0.0634, Val Loss: 0.1446, Akurasi: 94.56%
Epoch 148/350, Loss: 0.0623, Val Loss: 0.1441, Akurasi: 94.56%
Epoch 149/350, Loss: 0.0612, Val Loss: 0.1436, Akurasi: 94.56%
Epoch 150/350, Loss: 0.0601, Val Loss: 0.1432, Akurasi: 94.56%
Epoch 151/350, Loss: 0.0591, Val Loss: 0.1427, Akurasi: 94.56%
Epoch 152/350, Loss: 0.0580, Val Loss: 0.1423, Akurasi: 94.56%
Epoch 153/350, Loss: 0.0570, Val Loss: 0.1420, Akurasi: 94.56%
Epoch 154/350, Loss: 0.0560, Val Loss: 0.1416, Akurasi: 94.33%
Epoch 155/350, Loss: 0.0551, Val Loss: 0.1413, Akurasi: 94.33%
Epoch 156/350, Loss: 0.0541, Val Loss: 0.1409, Akurasi: 94.09%
Epoch 157/350, Loss: 0.0532, Val Loss: 0.1406, Akurasi: 94.09%
Epoch 158/350, Loss: 0.0523, Val Loss: 0.1402, Akurasi: 94.09%
Epoch 159/350, Loss: 0.0514, Val Loss: 0.1399, Akurasi: 94.33%
Epoch 160/350, Loss: 0.0506, Val Loss: 0.1396, Akurasi: 94.33%
Epoch 161/350, Loss: 0.0497, Val Loss: 0.1393, Akurasi: 94.33%
Epoc